# Linear Regression

After exploring the Exoplanet dataset, I noticed that a considerable number of planets were lacking data points for their effective temperature and inclination, with 1396 and 1286 planets missing each, respectively.

While linear regression method can be used for filling in missing data, it should be done with caution when a multitude of other variables also have missing values. This is because correlations between variables can become unstable and result in further complications.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math

df = pd.read_csv('exoplanet_dataset.csv')

# Drop the uncertainties and limit columns to simplify the data
df = df.drop([col_name for col_name in df if col_name.endswith(('err1', 'err2', 'lim'))], axis=1)

# Update the data type int64 into bool on boolean columns as documented in the column definition
bool_col_names = ['rv_flag', 'pul_flag', 'ptv_flag', 'tran_flag', 'ast_flag', 'obm_flag', 'micro_flag', 'etv_flag', 'ima_flag', 'dkin_flag', 'ttv_flag', 'cb_flag']
df[bool_col_names] = df[bool_col_names].astype(bool)

col_to_remove = list((df.isnull().sum() / len(df)).loc[lambda p : p > 0.3].sort_values(ascending=False).index) # Drop columns with missing data >= 30%
df = df.drop(col_to_remove, axis=1)
df = df.drop(['pl_rade', 'pl_bmasse'], axis=1) # Drop highly correlated value
df = df.drop(['discoverymethod'], axis=1) # Use one-hot encoded columns
df = df.drop(['pl_name', 'hostname', 'disc_year', 'disc_locale', 'disc_facility', 'disc_telescope', 'disc_instrument', 'pl_bmassprov', 'st_metratio', 'ra_reflink'], axis=1) # Drop non numeric fields
df = df.dropna() # Drop nan values

## Split dataset into Train and Test

Among the planets that have the temperature data, split the dataset into two groups: train(70%) and test(30%).

In [2]:
from sklearn.model_selection import train_test_split

temp_df = df[df['pl_eqt'].notnull()]
temp_pred_df = df[df['pl_eqt'].isnull()]
y = temp_df['pl_eqt']
X = temp_df.drop('pl_eqt', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # random state is used to get the same output each time

# Convert to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (1993, 58)
y_train shape: (1993,)
X_test shape: (855, 58)
y_test shape: (855,)


## Compute cost

The model function for linear regression which is a function that maps from `X` (planet parameters) to `y` (planet effective temperature (K)) is represented as: $$f_{W,b}(X) = WX + b$$
To train this linear regression model, you need to find the best $(W,b)$ parameters that fit your dataset.
To compare how a one parameter $(W,b)$ is better or worse than another is by evaluating it with a cost function $J(W,b)$.


The cost function is defined as: $$J(W,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{W,b}(X^{(i)}) - y^{(i)})^2$$

where $m$ is the number of training examples, $f_{W,b}(X^{(i)})$ is the model's prediction of the planet's effective temperature and $y^{(i)}$ is the actual effective temperature from the dataset.

In [3]:
def compute_cost(X, y, W, b): 
    """
    Computes the cost function for multi-variable linear regression.
    
    Arguments:
    X: input matrix of shape (m, n) (Planet parameters) 
    y: output vector of shape (m, 1) (Actual effective temperature of the planet)
    W: weight vector of shape (1, n)
    b: bias scalar
    
    Returns
    cost -- scalar value of the cost function
    """
    
    m = X.shape[0] # number of training examples
    f_Wb = np.dot(X, W.T) + b  # predicted output values
    loss = (f_Wb - y)**2  # squared error 
    cost = np.sum(loss) / (2 * m)  # compute cost using mean squared error

    return cost

## Gradient descent

The parameter that fits the data best will have the smallest cost $J(W,b)$. Gradient descent is used to find that smallest cost by stepping closer to the optimal value. 
The algorithm for gradient descent is:
$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline\; 
& \phantom {0000} b := b -  \alpha \frac{\partial J(W,b)}{\partial b} \newline\; 
& \phantom {0000} W := W -  \alpha \frac{\partial J(W,b)}{\partial W} \; \newline 
& \rbrace\end{align*}$$

where parameters $W$, $b$ are both updated simultaniously and where:
$$\frac{\partial J(W,b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{W,b}(X^{(i)}) - y^{(i)})$$

$$\frac{\partial J(W,b)}{\partial W}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{W,b}(X^{(i)}) -y^{(i)})X^{(i)}$$

In [4]:
def compute_gradient(X, y, W, b): 
    """
    Compute the gradient of the cost function for multi-variable linear regression.

    Arguments:
    X: input matrix of shape (m, n) (Planet parameters) 
    y: output vector of shape (m, 1) (Actual effective temperature of the planet)
    W: weight vector of shape (1, n)
    b: bias scalar
    
    Returns
    dj_dW: The gradient of the cost with respect to the parameters W
    dj_db: The gradient of the cost with respect to the parameter b     
    """
    
    m = X.shape[0]  # number of training examples
    f_Wb = np.dot(X, W.T) + b  # predicted output values
    diff = f_Wb - y  # difference between predicted and true output values

    dj_dW = np.dot(diff.T, X) / m  # gradient with respect to the weights
    dj_db = np.sum(diff) / m  # gradient with respect to the bias

    return dj_dW, dj_db

## Batch gradient descent

In [5]:
def gradient_descent(X, y, W_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent

    Arguments:
    X: input matrix of shape (m, n)
    y: output vector of shape (m, 1)
    W_in: initial weight vector of shape (1, n)
    b_in: initial bias scalar
    cost_function: function to compute the cost given X, y, W, and b
    gradient_function: function to compute the gradient of the cost function given X, y, W, and b
    alpha: learning rate
    num_iters: number of iterations to run gradient descent

    Returns:
    W: optimal weight vector of shape (1, n)
    b: optimal bias scalar
    """

    # An array to store cost J and w's at each iteration — primarily for graphing later
    J_history = []
    W_history = []
    W = copy.deepcopy(W_in)  # Avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_dW, dj_db = gradient_function(X, y, W, b)  

        # Update Parameters using w, b, alpha and gradient
        W = W - alpha * dj_dW               
        b = b - alpha * dj_db               

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost = cost_function(X, y, W, b)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0:
            W_history.append(W)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return W, b, J_history, W_history # Return W and J,W history for graphing

In [ ]:
m,n = X.shape 

# initialize parameters
initial_W = np.random.randn(1, n) * 0.000005
initial_b = 0

# gradient descent settings
iterations = 1500
alpha = 0.001

W,b,J_history,W_history = gradient_descent(X_train, y_train, initial_W, initial_b, compute_cost, compute_gradient, alpha, iterations)
print("W,b found by gradient descent:", W, b)

Iteration    0: Cost 37821222289488555579822571520.00   
